<a href="https://colab.research.google.com/github/eyesimk/YouDo/blob/main/YouDo2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [52]:
import pandas as pd
from tqdm import trange
import numpy as np

In [53]:
df = pd.read_csv('https://files.grouplens.org/datasets/movielens/ml-100k/u.data', delimiter=r'\t',
names=['user_id', 'item_id', 'rating', 'timestamp'])

/usr/local/lib/python3.7/dist-packages/pandas/util/_decorators.py:311: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  return func(*args, **kwargs)


In [54]:
df

,user_id,item_id,rating,timestamp
0,196,242,3,881250949
1,186,302,3,891717742
2,22,377,1,878887116
3,244,51,2,880606923
4,166,346,1,886397596
...,...,...,...,...
99995,880,476,3,880175444
99996,716,204,5,879795543
99997,276,1090,1,874795795
99998,13,225,2,882399156


In [55]:
r = df.pivot(index='user_id', columns='item_id', values='rating').values
r

array([[ 5.,  3.,  4., ..., nan, nan, nan],
       [ 4., nan, nan, ..., nan, nan, nan],
       [nan, nan, nan, ..., nan, nan, nan],
       ...,
       [ 5., nan, nan, ..., nan, nan, nan],
       [nan, nan, nan, ..., nan, nan, nan],
       [nan,  5., nan, ..., nan, nan, nan]])

In [56]:
r.shape

(943, 1682)

In [57]:
r_copy = r.copy()


In [58]:
irow, jcol = np.where(~np.isnan(r))

In [59]:
y_pred= np.zeros(r.shape)

**PART 1**

In [60]:
def fit(r, iter=100, l_rate= 0.0025):

  b_user = np.zeros(r.shape[0])
  b_item = np.zeros(r.shape[1]) 

  for i in range(iter):
 
    for u, j in zip(irow, jcol):
        y_pred[u][j] = b_user[u] + b_item[j]
    
    loss = 0.5 * sum(((y_pred[irow,jcol] - r[irow,jcol]) ** 2 ))

    g = - np.nansum(np.dstack((r,-y_pred)),2)
    
    b_item -= np.nansum(g, axis=0) * l_rate
    b_user -= np.nansum(g, axis=1) * l_rate

  print(loss)
  return b_item, b_user
  

fit(r)



41939.404817357834


(array([1.89981225, 1.2924681 , 1.08494463, ..., 0.06404272, 0.34704024,
        0.27600141]),
 array([2.02481121, 2.04819338, 1.4651994 , 2.86743212, 1.55878041,
        1.72414097, 2.39110603, 2.1787096 , 2.41003199, 2.32597193,
        1.87864044, 2.51111347, 1.70277785, 2.18357078, 1.45835376,
        2.56180692, 1.37473312, 2.02428591, 1.80485969, 1.44150264,
        1.62760485, 1.97222922, 1.8455099 , 2.50804436, 2.09912912,
        1.50229433, 1.75213138, 2.08786843, 2.03093613, 2.30744765,
        2.07363111, 1.77009914, 2.36882866, 2.7036673 , 1.88159606,
        2.60222667, 2.16753123, 2.48992792, 2.12112427, 1.35191074,
        1.77633294, 2.14456378, 2.13360111, 1.96600648, 2.27147143,
        2.3961573 , 2.09732341, 1.90172326, 1.26294905, 1.92813155,
        1.52352163, 2.5506003 , 2.15570677, 2.2872568 , 1.66512716,
        2.18473216, 2.14824441, 2.01625944, 2.26214518, 2.23965065,
        1.55141849, 1.66708452, 1.57307754, 1.84003986, 2.10199578,
        1.91563839, 2

PART **2**

In [61]:
def fit_reg(r, iter=100, l_rate= 0.0025, lam = 0.00001):

  b_user = np.zeros(r.shape[0])
  b_item = np.zeros(r.shape[1]) 

  for i in range(iter):
 
    for u, j in zip(irow, jcol):
        y_pred[u][j] = b_user[u] + b_item[j]


    reg_term = lam * 0.5 * (sum(b_user**2) + sum(b_item**2))
    
    loss = 0.5 * sum(((y_pred[irow,jcol] - r[irow,jcol]) ** 2 )) + reg_term

    g0 = -1 * np.nansum(np.dstack((r,-y_pred)),2) + np.nansum(lam * b_item)
    g1 = -1 * np.nansum(np.dstack((r,-y_pred)),2) + np.nansum(lam * b_user)
    
    b_item -= np.nansum(g0, axis=0) * l_rate
    b_user -= np.nansum(g1, axis=1) * l_rate
    
  print(loss)
  return b_item, b_user
  

fit_reg(r)



51668.264824813625


(array([ 2.51585801,  1.62661317,  1.43441183, ..., -2.30867076,
        -2.05317111, -2.11823463]),
 array([ 1.64695854e+00,  1.23050358e+00,  5.99101700e-01,  1.62005459e+00,
         1.22837923e+00,  1.22005396e+00,  2.09659543e+00,  1.40929255e+00,
         1.12620627e+00,  1.87304479e+00,  1.42159469e+00,  1.69908172e+00,
         1.50301065e+00,  1.55902742e+00,  8.21408361e-01,  2.03047564e+00,
         1.38519282e-01,  1.64451872e+00,  3.56005649e-01,  5.74584856e-01,
         1.34236749e+00,  1.52001440e+00,  1.32492299e+00,  1.76191805e+00,
         1.43745787e+00,  8.09264094e-01,  5.46471068e-01,  1.44137641e+00,
         9.13969061e-01,  1.33748583e+00,  1.11984982e+00,  7.71021797e-01,
         9.54053447e-01,  1.26462033e+00,  5.29970951e-01,  1.25748032e+00,
         1.45611068e+00,  2.08375665e+00,  6.39115640e-01,  1.91062039e-01,
         9.59332050e-01,  1.68115660e+00,  1.63822474e+00,  1.41241513e+00,
         1.44409530e+00,  1.15288137e+00,  7.13316783e-01,  1.1